In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Sensitivity to ETI RADAR

If we would like to determine the sensitivity to a ETI RADAR system we have to consider the intensity of, and distance to the ETI signal, and the sensitivity of our radio telescopes. We need to answer the following:

1. What is the flux density of a RADAR or any broadband, short duration in time pulse from a distance source, such as ETI?
2. How sensitive would be to such a signal with the current and future radio telescopes?

We will use the Arecibo RADAR system as a proxy for an ETI RADAR system. The Arecibo S-band (12.6 cm, 2380 MHz) RADAR system ([1](http://www.naic.edu/~nolan/radar/),[2](http://www.naic.edu/~astro/RXstatus/Sradar/Sradar.shtml)) is a highly-directive, long-distance signal marker for what ETI might use to announce their presence. It has the following specifications:

| S-Band RADAR  |               |
| -------------:| ------------- |
| Transmission Frequency | 2380 MHz |
| Bandwidth     | 100 MHz       |
| Polarization  | Dual Circular |
| Power         | 900 kW ([3](https://arxiv.org/pdf/1604.01080.pdf)) |
| Gain          | 10 K/Jy ([4](http://www.naic.edu/~phil/pnt/pnterrs_sysperf_afterPainting_apr16.html),[5](http://www.naic.edu/~phil/pnt/sbnx102_160421.pdf)) |
| $T_{\textrm{sys}}$ | 25 K ([4](http://www.naic.edu/~phil/pnt/pnterrs_sysperf_afterPainting_apr16.html),[5](http://www.naic.edu/~phil/pnt/sbnx102_160421.pdf)) |

Using the Nyquist approximation for a thermal noise source ([6](https://en.wikipedia.org/wiki/Johnson%E2%80%93Nyquist_noise)), the power of the RADAR can be related to a temperature:

$$P_{\textrm{RADAR}} = k \cdot T_{\textrm{RADAR}} \cdot \Delta \nu$$

where $k$ is the Boltzman constant and $\Delta \nu$ is the bandwidth.

In [ ]:
pRadar = 9e5 # Watts (J/s)
bw = 100e6 # Hz (1/s)
kk = 1.38064852e-23 # Boltzman constant J/K
tRadar = pRadar / (kk * bw) # K

### Flux Density of the Arecibo RADAR

As an approximation we can use the angular resolution of the Arecibo dish as the effective half-power beam width. That is, half of the RADAR power is transmitted within a given solid angle on the sky.

$$\theta_{\textrm{HPBW}} = 1.22 \frac{\lambda}{D} = 1.22 \frac{c}{\nu \cdot D}$$

where $D = 305$ m is the diameter of Arecibo, though the effective diameter is smaller because the spheriodal primary dish is only partially illuminated.

$$\Omega_{\textrm{Beam}} = \frac{\pi \theta_{\textrm{HPBW}}^2}{2}$$

In [ ]:
cc = 299792458.0 # m/s
areciboDiameter = 305. # m
fCenter = 2380e6 # Hz
freqs = fCenter + np.array([-1. * bw/2., bw/2.]) # lowest and highest frequency
thetaHPBW = 1.22 * cc / (areciboDiameter * freqs) # radians
omegaBeam = np.pi * thetaHPBW / 2. # steradians
avgOmegaBeam = np.mean(omegaBeam)

print 'theta_HPBW: %f arcmin'%(np.mean(thetaHPBW) * (180./np.pi) * 60.)
print 'Omega_beam: %f sr'%avgOmegaBeam

We can compute an 'apparent' Luminosity assuming the RADAR is coming from an isotropic source. Of course this is not correct, as the dish geometry of Arecibo makes creates a highly directional beam. But, we will compute this apparent luminosity to compute the received flux. We do this using the assumption that the RADAR transmitter is aligned at boresight with the receiving telescope.

$$L_{\textrm{RADAR, iso}, \nu} = \frac{4 \pi}{\Omega_{\textrm{Beam}}} \frac{P_{\textrm{RADAR}}}{\Delta \nu}$$

In [ ]:
lumIso = (4. * np.pi / avgOmegaBeam) * ( pRadar / bw) # isotropic luminsity, W/Hz#
print 'Luminosity: %f W/Hz'%lumIso

The flux density of a source $S_{\nu}$ can be related to the isotropic luminsity as

$$S_{\nu} = \frac{L_{\textrm{iso}, \nu}}{4 \pi r^2}$$

where $r$ is the distance from the source to the detector. The flux density is in units $\textrm{W} \; \textrm{m}^{-2} \; \textrm{Hz}^{-2}$ and can be converted to Janskys (Jy) with

$$1 \; \textrm{Jy} = 10^{-26} \; \frac{\textrm{W}}{\textrm{m}^2 \textrm{Hz}}$$

In [ ]:
fluxDen = lambda r: lumIso / (4. * np.pi * r**2.)
fluxDenJy = lambda r: fluxDen(r) / 10e-26

ly2m = 9460730472580800. # number of meters in a light year
distMoonEarth = 384400000. # distance from Earth to Moon in meters
distJupiterEarth = 588e9 # distance from Earth to Jupiter in meters
distAlphaCentEarth = 4.367 * ly2m
print 'Flux density of the Arecibo RADAR from the Moon: %f Jy'%fluxDenJy(distMoonEarth)
print 'Flux density of the Arecibo RADAR from Jupiter: %f Jy'%fluxDenJy(distJupiterEarth)
print 'Flux density of the Arecibo RADAR from Alpha Centari: %f nJy'%(1e9*fluxDenJy(distAlphaCentEarth))

We can see that even out to near by stars the flux density of the Arecibo RADAR is very small. But, all is not lost, we have to consider the sensitivity of the receiving system.

---

### Telescope Sensitivity

The minimum sensitivity of a radio telescope $S_{\textrm{min}}$ is described by the *radiometer equation*, for the case of a RADAR source it has been slightly expanded

$$S_{\textrm{min}} = \textrm{SEFD} \frac{\textrm{SNR}_\textrm{min}}{N_{\textrm{antenna}} \; \sqrt{D \; N_{\textrm{pol}} \; \Delta \tau_{\textrm{obs}} \; \Delta \nu_{\textrm{obs}}}}$$

where $\textrm{SEFD}$ is the system-equivalent flux density of a telescope in an array of $N_{\textrm{antenna}}$ antennas with $N_{\textrm{pol}}$ polarizations, for a given integration time $\Delta \tau_{\textrm{obs}}$ and bandwidth $\Delta \nu_{\textrm{obs}}$. The last term $D$ is the duty cycle of the RADAR, for a continuous signal $D=1$, if the RADAR has a pulse width of 1 ms, every second then $D = \frac{1 \textrm{ms}}{1 \textrm{s}} = 0.001$. Note, we are assuming $\Delta \tau_{\textrm{obs}}$ is a longer time length then the RADAR repetition time scale.

Now assuming we know the duty cycle rate and bandwidth of the transmitting RADAR, how far away is a radio telescope sensitive to? Note, these assumptions are for the optimal detection case. Performing a large matched filter search over the time, duty cycle, and bandwidth space it is reasonable to make these assumptions.

For our test case we will use $D=1$, a minimum detection SNR of 7 and the Arecibo S-band RADAR specifications to look at the sensitivity of Arecibo, GBT, Parkes, MeerKAT, and the SKA. Note, we are assuming that an L-band RADAR would have similar specification for these telescopes.

In [ ]:
dutyCycle = 1. # duty cycle of RADAR
snrMin = 7. # minimum SNR needed for verifiable detection
nuObs = 100e6
minObsTime = np.log10(0.001)
maxObsTime = np.log10(10000.)
tObs = np.logspace(minObsTime, maxObsTime, num=1024) # observation integration time range
print tObs

def sMin(SEFD, snr, nAnt, nPol, dc, tObs, nuObs):
    """
    SEFD: float, system-equivalent flux density
    snr: float, minimum SNR needed for verifiable detection
    nAnt: int, number of antennas/elements in the telescope array
    nPol, int, number of polarizations in the reciever (1 or 2)
    dc: float, duty cycle of RADAR
    tObs: float, observational integration time, in seconds
    nuObs: float, observational bandwidth, in Hz
    """
    return SEFD * snr / (nAnt * np.sqrt(dc * nPol * tObs * nuObs))

#### Arecibo

Using the L-band system, the SEFD is approximately 3 Jy ([7](http://www.naic.edu/alfa/performance/)).

In [ ]:
areciboSEFD = 3. # Jy
areciboSMin = sMin(areciboSEFD, snrMin, 1, 2., dutyCycle, tObs, nuObs)

#### GBT

Using the L-band system, the SEFD is approximately 10 Jy ([8](https://science.nrao.edu/facilities/gbt/proposing/GBTpg.pdf)).

In [ ]:
gbtSEFD = 10. # Jy
gbtSMin = sMin(gbtSEFD, snrMin, 1, 2., dutyCycle, tObs, nuObs)

#### Parkes

Using the L-band system, using $T_{\textrm{sys}} = 25 \textrm{K}$ and gain $G = 1 / 1.2 \; \textrm{K/Jy}$, and $\textrm{SEFD} = T_{\textrm{sys}} / G$, then SEFD is approximately 30 Jy ([9](https://www.parkes.atnf.csiro.au/observing/documentation/user_guide/pks_ug_3.html)).

In [ ]:
parkesSEFD = 25. * 1.2 # Jy
parkesSMin = sMin(parkesSEFD, snrMin, 1, 2., dutyCycle, tObs, nuObs)

#### MeerKAT

Using the L-band system, the SEFD is approximately 440 Jy ([10](http://public.ska.ac.za/meerkat/meerkat-schedule)).

In [ ]:
meerkatSEFD = 440. # Jy
meerkatSMin = sMin(meerkatSEFD, snrMin, 64, 2., dutyCycle, tObs, nuObs)

#### SKA Phase 1

The SEFD of a dish element is approximately that of a MeerKAT dish. In Phase 1 250 dishes are planned ([11](file:///home/griffin/Downloads/21705_130_Memo_Dewdney.pdf)).

In [ ]:
ska1SEFD = 440. # Jy
ska1SMin = sMin(ska1SEFD, snrMin, 250, 2., dutyCycle, tObs, nuObs)

#### SKA Phase 2

In phase 2 the SEFD of a dish element not expected to chang, 2000 dishes are planned.

In [ ]:
ska2SEFD = 440. # Jy
ska2SMin = sMin(ska2SEFD, snrMin, 2000, 2., dutyCycle, tObs, nuObs)

In [ ]:
plt.loglog(tObs, areciboSMin, label='Arecibo')
plt.loglog(tObs, gbtSMin, label='GBT')
plt.loglog(tObs, parkesSMin, label='Parkes')
plt.loglog(tObs, meerkatSMin, label='MeerKAT')
plt.loglog(tObs, ska1SMin, label='SKA Phase 1')
plt.loglog(tObs, ska2SMin, label='SKA Phase 2')
plt.ylabel('Flux Density (Jy)')
plt.xlabel('Integration Time (s)')
plt.xlim(10**minObsTime, 10**maxObsTime)
#plt.legend()

---

The gain of an antenna is a product of the antenna efficiency $E_{\textrm{antenna}}(\nu)$ and the antenna directivity $D_{\textrm{antenna}}(\theta, \phi, \nu)$

$$G_{\textrm{antenna}}(\theta, \phi, \nu) = E_{\textrm{antenna}}(\nu) \cdot D_{\textrm{antenna}}(\theta, \phi, \nu)$$

where $\nu$ is the observing/transmission frequency, and $(\theta, \phi)$ is the azimuth and altitude relative to boresight. For Arecibo the gain at boresight has been measured to be approximately 10 K/Jy. We will assume we are considering a transmission in which a RADAR system is pointed directly at the target. This is an upper limit on the detectability of a RADAR signal.

$$G_{\textrm{Arecibo, measured}} (0,0, 2380 \; \textrm{MHz}) \approx 10 \; \textrm{K/Jy}$$


In [ ]:
gainArecibo = 10. # K/Jy
print tRadar / gainArecibo , 'Jy'

In [ ]:
# Arecibo planetary radar bandwidth, power, time duration
# sensitivity (radiometer eq.) of arecibo, gbt, parkes, meerkat, ska 1, ska 2
# number of stars sensitive to based on radar parameters